# Running a program remotely

In this tutorial, we will write a basic program using Quantum Serverless. We will show how to run the program remotely and retrieve the results from the serverless client.

### Writing the Program

First, we need to write the program code and save it to a file called [program.py](./source_files/program.py). This program creates a two-qubit quantum circuit that prepares a Bell state, measures the result, and saves the measured probability distribution.

The code for the program is shown below:

```python
# source_files/program.py

from qiskit import QuantumCircuit
from qiskit.primitives import Sampler

from quantum_serverless import save_result

# all print statement will be available in job logs
print("Running program 1...")

# creating circuit
circuit = QuantumCircuit(2)
circuit.h(0)
circuit.cx(0, 1)
circuit.measure_all()

# running Sampler primitive
sampler = Sampler()
quasi_dists = sampler.run(circuit).result().quasi_dists

# save results of program execution, 
# which will be accessible by calling `.result()`
save_result(quasi_dists)
print("Completed running program 1.")
```

### Running the Program

To run the program, we need to import the necessary classes and configure them. One of these classes is QuantumServerless, which is a client class for interacting with compute resources.

QuantumServerless takes a [BaseProvider](https://qiskit-extensions.github.io/quantum-serverless/stubs/quantum_serverless.core.BaseProvider.html) instance as a constructor argument. The provider stores configuration information about our compute resources, such as where they are located and how to connect to them. In this example, we will use a provider that is connected to a local Docker Compose setup. In this case, it allows us to run the program locally on our machine. If you want to run the program elsewhere, you will need to provide the corresponding host and authentication details.

In [1]:
from quantum_serverless import QuantumServerless, ServerlessProvider
import os

> &#x26A0; This provider is set up with default credentials to a test cluster intended to run on your machine. For information on setting up infrastructure on your local machine, check out the guide on [local infrastructure setup](https://qiskit-extensions.github.io/quantum-serverless/deployment/local.html).

In [2]:
provider = ServerlessProvider(
    username=os.environ.get("GATEWAY_USER", "user"),
    password=os.environ.get("GATEWAY_PASSWORD", "password123"),
    # token=os.environ.get("GATEWAY_TOKEN", "<TOKEN>"), # token can be used instead of user/password combination
    host=os.environ.get("GATEWAY_HOST", "http://localhost:8000"),
)

serverless = QuantumServerless(provider)
serverless

<QuantumServerless | providers [gateway-provider]>

After importing the necessary classes and configuring them, we can run the program by calling the `run()` method of the [QuantumServerless](https://qiskit-extensions.github.io/quantum-serverless/stubs/quantum_serverless.QuantumServerless.html#quantum_serverless.QuantumServerless) object:

[Program](https://qiskit-extensions.github.io/quantum-serverless/stubs/quantum_serverless.core.Program.html#quantum_serverless.core.Program) accepts couple of required parameters:

- title - name of the program
- entrypoint - name of python file you want to execute
- working_dir - directory where  your script is located (directory size must be less than 50MB). This is optional parameter and will be current folder by default.

> Warning! All content of `working_dir` will be shipped to cluster for execution

In [3]:
from quantum_serverless import Program

program = Program(
    title="First program", entrypoint="program.py", working_dir="./source_files/"
)

job = serverless.run(program)
job

<Job | e13dac4b-6d5b-4616-896d-42881371876e>

[Job](https://qiskit-extensions.github.io/quantum-serverless/stubs/quantum_serverless.core.Job.html#quantum_serverless.core.Job) instances have a `status()` method to check status of program execution.

In [4]:
job.status()

'QUEUED'

`Job` instances also have a `result()` method for retrieving results. The `result()` method will not return until the job is done running the program.

In [5]:
job.result()

'[{"0": 0.4999999999999999, "3": 0.4999999999999999}]'

To inspect the logs from a program, access them from the ``Job`` instance.

In [6]:
print(job.logs())

OpenBLAS WARNING - could not determine the L2 cache size on this system, assuming 256k
OpenBLAS WARNING - could not determine the L2 cache size on this system, assuming 256k
Running program 1...
Completed running program 1.



`QuantumServerless` object has method `.widget` which renders Jupyter widget to see list of executed programs.

![widget](./images/widget.png)

In [7]:
serverless.widget()